In [1795]:
import pandas as pd
import numpy as np
import holidays
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score, accuracy_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
from plotly.subplots import make_subplots


In [1796]:
filepath ='Walmart_Store_sales.csv'

In [1797]:
df=pd.read_csv(filepath)
df.shape, df.head(), display(df),

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092
...,...,...,...,...,...,...,...,...
145,14.0,18-06-2010,2248645.59,0.0,72.62,2.780,182.442420,8.899
146,7.0,NaN,716388.81,NaN,20.74,2.778,NaN,NaN
147,17.0,11-06-2010,845252.21,0.0,57.14,2.841,126.111903,NaN
148,8.0,12-08-2011,856796.10,0.0,86.05,3.638,219.007525,NaN


((150, 8),
    Store        Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
 0    6.0  18-02-2011    1572117.54           NaN        59.61       3.045   
 1   13.0  25-03-2011    1807545.43           0.0        42.38       3.435   
 2   17.0  27-07-2012           NaN           0.0          NaN         NaN   
 3   11.0         NaN    1244390.03           0.0        84.57         NaN   
 4    6.0  28-05-2010    1644470.66           0.0        78.89       2.759   
 
           CPI  Unemployment  
 0  214.777523         6.858  
 1  128.616064         7.470  
 2  130.719581         5.936  
 3  214.556497         7.346  
 4  212.412888         7.092  ,
 None)

In [1798]:
df.describe(include='all')

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


In [1799]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         150 non-null    float64
 1   Date          132 non-null    object 
 2   Weekly_Sales  136 non-null    float64
 3   Holiday_Flag  138 non-null    float64
 4   Temperature   132 non-null    float64
 5   Fuel_Price    136 non-null    float64
 6   CPI           138 non-null    float64
 7   Unemployment  135 non-null    float64
dtypes: float64(7), object(1)
memory usage: 9.5+ KB


In [1800]:
df['Store'] = df['Store'].astype(int)


In [1801]:

df.isna().sum()

Store            0
Date            18
Weekly_Sales    14
Holiday_Flag    12
Temperature     18
Fuel_Price      14
CPI             12
Unemployment    15
dtype: int64

In [1802]:
def null_value(col):
    # Group by the 'Store' column and sum the number of missing values for each store
    df_null = df.groupby(col).apply(lambda group: group.isna().sum(), include_groups=False)

    # Group by the 'Store' column and sum the number of present values for each store
    df_value = df.groupby(col).apply(lambda group: group.notna().sum(), include_groups=False)

    # Iterate over the results and print the number of missing and present values for each store
    for store in df_null.index:
        null_counts = df_null.loc[store]
        value_counts = df_value.loc[store]
        total_null = null_counts.sum()
        total_value = value_counts.sum()
        ratio_null= round((total_null*100)/(total_value+total_null),2)
        print(f'le magasin {store} a {ratio_null}% de valeur manquante')

null_value('Store')


le magasin 1 a 12.99% de valeur manquante
le magasin 2 a 12.5% de valeur manquante
le magasin 3 a 9.52% de valeur manquante
le magasin 4 a 8.16% de valeur manquante
le magasin 5 a 11.11% de valeur manquante
le magasin 6 a 10.2% de valeur manquante
le magasin 7 a 8.93% de valeur manquante
le magasin 8 a 7.14% de valeur manquante
le magasin 9 a 10.71% de valeur manquante
le magasin 10 a 11.43% de valeur manquante
le magasin 11 a 14.29% de valeur manquante
le magasin 12 a 2.86% de valeur manquante
le magasin 13 a 3.17% de valeur manquante
le magasin 14 a 14.29% de valeur manquante
le magasin 15 a 10.71% de valeur manquante
le magasin 16 a 3.57% de valeur manquante
le magasin 17 a 17.86% de valeur manquante
le magasin 18 a 8.57% de valeur manquante
le magasin 19 a 4.76% de valeur manquante
le magasin 20 a 12.24% de valeur manquante


In [1803]:
# On supprime les ligne où la valeur de Weekly_Sales est absente
df= df.dropna(subset=['Weekly_Sales'],axis=0)

In [1804]:
df.isna().sum()

Store            0
Date            18
Weekly_Sales     0
Holiday_Flag    11
Temperature     15
Fuel_Price      12
CPI             11
Unemployment    14
dtype: int64

Contrôle des outliers

In [1805]:
# On crée une figure avec une grille de 2x3 pour afficher nos boxplot
fig = make_subplots(rows=2, cols=2, subplot_titles=('Temperature', 'Fuel Price', 'CPI', 'Unemployment'))

# on crée nos boxplot pour chaque variables
fig.add_trace(go.Box(y=df['Temperature'], name='Temperature'), row=1, col=1)
fig.add_trace(go.Box(y=df['Fuel_Price'], name='Fuel_price'), row=1, col=2)
fig.add_trace(go.Box(y=df['CPI'], name='CPI'), row=2, col=1)
fig.add_trace(go.Box(y=df['Unemployment'],name='Unemployment'), row=2, col=2)

fig.update_layout(title_text="Boxplot de la distribution")
fig.show()


Nous avons des outliers uniquement chez les Unemployement, nous allons les traiter.

In [1806]:
columns = ['Unemployment'] 

for c in columns:
    mean_col = df[c].mean()
    std_col = df[c].std()
    outliers = (df[c] < mean_col - 3*std_col)|(df[c] > mean_col + 3*std_col)
    df=df[~outliers]
    
    


In [1807]:
df.shape, df.isna().sum()

((131, 8),
 Store            0
 Date            18
 Weekly_Sales     0
 Holiday_Flag    11
 Temperature     14
 Fuel_Price      12
 CPI             11
 Unemployment    14
 dtype: int64)

Nous avons encore des valeurs manquantes. Nous pouvons facilement traiter les jours fériés grace à la bibliothèque 'holidays'.

In [1808]:
# On filtre les NaN de la colonne jour férié
mask_holiday = df['Holiday_Flag'].isna()
df_holiday = df[mask_holiday]
df_holiday

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
15,6,30-04-2010,1498080.16,NaN,68.91,2.780,211.894272,7.092
43,7,26-08-2011,629994.47,NaN,57.60,3.485,194.379637,8.622
48,1,05-08-2011,1624383.75,NaN,91.65,3.684,215.544618,7.962
53,14,25-03-2011,1879451.23,NaN,41.76,3.625,184.994368,8.549
65,10,NaN,1714309.90,NaN,43.43,3.287,127.191774,8.744
73,1,27-08-2010,1449142.92,NaN,85.22,2.619,211.567306,7.787
90,9,09-07-2010,485389.15,NaN,78.51,2.642,214.656430,6.442
118,9,18-06-2010,513073.87,NaN,82.99,2.637,215.016648,6.384
136,4,08-07-2011,2066541.86,NaN,84.59,3.469,129.112500,5.644


In [1809]:
# Récupération des jours fériés de 2010 et 2011
us_holiday=holidays.US(years=[2010, 2011]) 
df_holiday['Date'] = pd.to_datetime(df_holiday['Date'], format = "%d-%m-%Y")
df_holiday['Holiday']= df_holiday['Date'].dt.date.isin(us_holiday.keys())
df_holiday['Holiday']


0      False
15     False
43     False
48     False
53     False
65     False
73     False
90     False
118    False
136    False
146    False
Name: Holiday, dtype: bool

Nous n'avons pas de jour férié identifé nous mettons donc à 0

In [1810]:
df['Holiday_Flag'] = df['Holiday_Flag'].fillna(0)

In [1811]:
# Vérification des nouvelles valeurs
print(df['Holiday_Flag'].value_counts())
print('valeur manquante :', df['Holiday_Flag'].isna().sum())

Holiday_Flag
0.0    123
1.0      8
Name: count, dtype: int64
valeur manquante : 0


Pour ce qui est des dernières valeurs manquantes, nous allons supprimer les lignes avec uniquement les dates manquantes, pour les autres nous pourrons appliquer une méthode d'imputation lors du préprocessing. 

In [1812]:
df= df.dropna(subset=['Date'])

Nous créons des colonnes année, mois et jour.

In [1813]:
df['Date'] = pd.to_datetime(df['Date'], format = "%d-%m-%Y")
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df.shape
df.head(5)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,year,month,day
0,6,2011-02-18,1572117.54,0.0,59.61,3.045,214.777523,6.858,2011,2,18
1,13,2011-03-25,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011,3,25
4,6,2010-05-28,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010,5,28
5,4,2010-05-28,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010,5,28
6,15,2011-06-03,695396.19,0.0,69.80,4.069,134.855161,7.658,2011,6,3


# Analyse de la matrice de corrélation 

In [1814]:
# Corrélation matrix
corr = df.corr().round(2)
fig=px.imshow(corr, 
            labels=dict(x="Variables", y="Variables", color="Correlation"),
            x=corr.columns,
            y=corr.columns,
            color_continuous_scale='RdBu',
            text_auto=True)
fig.update_layout(width=800, height=800)  
# Ajoute les valeurs dans les cellules)
# fig = ff.create_annotated_heatmap(corr.values,
#                                   x = corr.columns.tolist(),
#                                   y = corr.index.tolist())
fig.show()

### Impact sur les ventes de la semaine:
- CPI corrélation négative: L'indice prix a un impact négatif sur les ventes de la semaine. Plus celui ci augmente plus cela aura un impact négatif sur le CA de la semaine. Les clients sont sensibles à l'inflation qui a un impact direct sur leur pouvoir d'achat
- Temperature corrélation négative: Les températures ont un impact modéré sur les ventes de la semaine. Certains types de magasins peuvent être sensibles à la météo
- Store corrélation positive: Certains magasins doivent réaliser plus de CA que d'autres 
Unemployment corrélation positive : Très étonnant plus il y a de chomeur et plus l'impact sur le CA est positif. Peut etre un phénomène local mais pas national.

### Corrélation entre variables
- Fuel Price/Date/Year : Le prix du carburant évolue à la hausse avec le temps. Forte corrélation de ces variables donc attention à leur utilisation, risque de redondance.
- CPI/Store : Des magasins sont plus sensilbes à la variation de l'indice prix, certains sont situés dans des secteurs où le cout de la vie est plus faible et d'autres plus élevé.

### Une faible corrélation avec les weekly_sales
De façon global il n'y a pas une seul varible explicative clair qui impacte le CA de la semaine. Il faudra s'appuyer sur plusieurs variables non linéaire afin de réaliser un modele de machine learning nous permettant de prédire les ventes de la semaines.
Il y a une très faible risque de mutlicolinéarité ( -0.3 et +0.3) des variables hors mis le Fuel/Date/Years où il faudra etre attentif à leur utilisation.

In [1815]:
fig = px.scatter(df, x='CPI', y='Weekly_Sales', trendline='ols')
fig.show()

- CPI corrélation négative: L'indice prix a un impact négatif sur les ventes de la semaine. Plus celui ci augmente plus cela aura un impact négatif sur le CA de la semaine. Les clients sont sensibles à l'inflation qui a un impact direct sur leur pouvoir d'achat.
Nous avons 3 clusters : Les magasins ayant un faible CPI se trouvent majoritairement dans une tranche de CA de 1 à 2M avec une variation à +/- 0.5M. Pour ceux ayant le plus fort CPI, les magasins ont des CA < 0.5M et 2M.Dans ce cluster il y a 2 groupes de CA magasins bien distinct ceux inférieur à 0.5M et ceux se situant entre 1.5M et 2M. Le troisieme cluster compte beaucoup moins de magasins et se décompose de la même manière que le magasins ayant un fort CPI avec un écart signifiquatif du CA entre 2 groupes de magasins.

In [1816]:
fig = px.scatter(df, x='Temperature', y='Weekly_Sales', trendline='ols')
fig.show()

- Temperature corrélation négative: Les températures ont un impact modéré sur les ventes de la semaine. Certains types de magasins peuvent être sensibles à la météo

In [1817]:
fig = px.scatter(df, x='Unemployment', y='Weekly_Sales', trendline='ols')
fig.show()

- Unemployment corrélation positive : Très étonnant plus il y a de chomeur et plus l'impact sur le CA est positif. Pas forcement étonnant cela dit étant donné que les gens ne travaillent pas, les restaurants, repas de midi au travail tout autant de consommation qui se font habituellement via des services exterieur se retrouvent à se faire à la maison. Pour exemple le COVID, certe vu a une autre échelle, montre bien des modes de consommation différents avec des magasins qui ont réalisé de très gros CA pendant cette période.

# PREPROCESSING

In [1818]:

target = "Weekly_Sales"
X = df.drop(target, axis=1)
Y = df.loc[:,target]

print(X, Y.head())


     Store       Date  Holiday_Flag  Temperature  Fuel_Price         CPI  \
0        6 2011-02-18           0.0        59.61       3.045  214.777523   
1       13 2011-03-25           0.0        42.38       3.435  128.616064   
4        6 2010-05-28           0.0        78.89       2.759  212.412888   
5        4 2010-05-28           0.0          NaN       2.756  126.160226   
6       15 2011-06-03           0.0        69.80       4.069  134.855161   
..     ...        ...           ...          ...         ...         ...   
144      3 2012-10-19           0.0        73.44       3.594  226.968844   
145     14 2010-06-18           0.0        72.62       2.780  182.442420   
147     17 2010-06-11           0.0        57.14       2.841  126.111903   
148      8 2011-08-12           0.0        86.05       3.638  219.007525   
149     19 2012-04-20           0.0        55.20       4.170  137.923067   

     Unemployment  year  month  day  
0           6.858  2011      2   18  
1          

In [1819]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=0)

In [1820]:
X_train.shape

(90, 10)

In [1821]:
numeric_features = ['Temperature','Fuel_Price', 'CPI','Unemployment', 'year', 'month', 'day']#[col for col in df.columns if col not in [target, 'Date', 'Holiday_Flag','Store']] #df.drop(target, axis=1).columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
categorical_features= ['Store', 'Holiday_Flag']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ])

In [1822]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num',  numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

model= LinearRegression()

In [1823]:
# Preprocessing sur le X_train
X_train= preprocessor.fit_transform(X_train)
print(X_train[0:2,:])
print('done...........')
# Preprocessing sur le X_test
X_test = preprocessor.transform(X_test)
print(X_test[0:2,:])

[[ 1.52824945e+00 -1.36747759e+00  9.86281451e-01 -9.67330408e-16
  -1.05558715e+00 -2.76032796e-02  1.01737744e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-4.16373351e-01 -8.77527971e-01 -1.15855666e+00  2.08608974e+00
  -1.05558715e+00  1.21454430e+00 -1.61052027e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]]
done...........
[[ 1.93531073e-01 -9.67028174e-16  9.25274539e-01 -7.0531

In [1824]:
model.fit(X_train, Y_train)


,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [1825]:
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

In [1826]:
# R2 Score 
print("R2score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2score on test set : ", r2_score(Y_test, Y_test_pred))

R2score on training set :  0.9727237843593463
R2score on test set :  0.9396190073986592


In [1827]:
# MAE
print("MAE on training set : ", mean_absolute_error(Y_train, Y_train_pred))
print("MAE on test set : ", mean_absolute_error(Y_test, Y_test_pred))

MAE on training set :  83585.16277908167
MAE on test set :  125016.44563683988


# Régularisation et optimisation des hyperparamètres:
####     > Ridge = Minimise la MSE en ajustant une pénalité sur les coefficients, tout en conservant toutes les variables (aucun coefficient n'est nul)
####      > Lasso = Minimise la MSE en ajustant une pénalité sur les coefficients, jusqu'à supprimer certaines variables (certains coefficients deviennent nuls).

In [ ]:
# Modèle Ridge simple
regressor_ridge = Ridge()
regressor_ridge.fit(X_train, Y_train)
print('R2score on training set :', regressor_ridge.score(X_train,Y_train))
print('R2score on test set : ', regressor_ridge.score(X_test, Y_test))

R2score on training set : 0.93944472421982
R2score on test set :  0.9239670316161102


In [ ]:
# Modifions l'alpha du Ridge sur 2 nouveaux fit afin de voir quel resultat nous avons
regressor_ridge1 = Ridge(alpha=10)
regressor_ridge1.fit(X_train, Y_train)
print('R2score on training set :', regressor_ridge1.score(X_train,Y_train))
print('R2score on test set : ', regressor_ridge1.score(X_test, Y_test))

R2score on training set : 0.5889770805776278
R2score on test set :  0.5916070839574004


In [1830]:
regressor_ridge2 = Ridge(alpha=1000)
regressor_ridge2.fit(X_train, Y_train)
print('R2score on training set :', regressor_ridge2.score(X_train,Y_train))
print('R2score on test set : ', regressor_ridge2.score(X_test, Y_test))

R2score on training set : 0.0314966140883689
R2score on test set :  0.021687658914468222


In [1831]:
# Récupération des noms des features
feature_names = preprocessor.get_feature_names_out()

In [1832]:
# Création d'un DataFrame reprennant nos coef pour différents alpha
data_dict = {
    'Features': feature_names,
    'Ridge': regressor_ridge.coef_,
    'Ridge1': regressor_ridge1.coef_,
    'Ridge2': regressor_ridge2.coef_
            }
recap_ridge = pd.DataFrame(data=data_dict)
recap_ridge

,Features,Ridge,Ridge1,Ridge2
0,num__Temperature,-38883.839378,-66582.586837,-9513.840956
1,num__Fuel_Price,-27777.076897,3524.944583,2029.721889
2,num__CPI,-147906.524503,-140210.285654,-15541.903863
3,num__Unemployment,-3226.071456,37312.789704,6560.630462
4,num__year,9998.363457,-19635.783190,-2222.298735
5,num__month,67409.386560,41049.693191,781.792366
6,num__day,-32635.456565,-13224.316041,-1751.546013
7,cat__Store_1.0,359638.366249,163336.201494,1824.104985
8,cat__Store_2.0,650358.714480,255439.993087,3355.829754
9,cat__Store_3.0,-661713.402504,-331241.713230,-7706.440446


Ridge ne fait pas de sélection de variables (contrairement au Lasso) → il ne met jamais exactement un coefficient à 0, mais réduit tous les coefficients vers zéro quand alpha augmente. Il applique une réduction uniforme de l’importance des features.

Plus l'alpha augmente et plus notre modèle devient moins performant 


In [1833]:
# Même process que Ridge avec 3 alpha différents et création du Dataframe Coef
regressor_lasso = Lasso(max_iter=100000)
regressor_lasso.fit(X_train, Y_train)
print('R2score on training set :', regressor_lasso.score(X_train,Y_train))
print('R2score on test set : ', regressor_lasso.score(X_test, Y_test))

R2score on training set : 0.9727237827081738
R2score on test set :  0.939624170537073


In [1834]:
regressor_lasso1 = Lasso(alpha=5, max_iter=1000000)
regressor_lasso1.fit(X_train, Y_train)
print('R2score on training set :', regressor_lasso1.score(X_train,Y_train))
print('R2score on test set : ', regressor_lasso1.score(X_test, Y_test))

R2score on training set : 0.9727237431566603
R2score on test set :  0.9396458780475373


In [1835]:
regressor_lasso2 = Lasso(alpha=10,  max_iter=100000)
regressor_lasso2.fit(X_train, Y_train)
print('R2score on training set :', regressor_lasso2.score(X_train,Y_train))
print('R2score on test set : ', regressor_lasso2.score(X_test, Y_test))

R2score on training set : 0.9727236199497183
R2score on test set :  0.9396758785782592


In [1836]:
data_dict = {
    'Features': feature_names,
    'Lasso': regressor_lasso.coef_,
    'Lasso1': regressor_lasso.coef_,
    'Lasso2': regressor_lasso.coef_
            }
recap_lasso = pd.DataFrame(data=data_dict)
recap_lasso

,Features,Lasso,Lasso1,Lasso2
0,num__Temperature,-3.150660e+04,-3.150660e+04,-3.150660e+04
1,num__Fuel_Price,-4.128519e+04,-4.128519e+04,-4.128519e+04
2,num__CPI,-9.553405e+04,-9.553405e+04,-9.553405e+04
3,num__Unemployment,-7.192310e+04,-7.192310e+04,-7.192310e+04
4,num__year,4.277918e+03,4.277918e+03,4.277918e+03
5,num__month,7.466588e+04,7.466588e+04,7.466588e+04
6,num__day,-3.797510e+04,-3.797510e+04,-3.797510e+04
7,cat__Store_1.0,2.749943e+05,2.749943e+05,2.749943e+05
8,cat__Store_2.0,6.308184e+05,6.308184e+05,6.308184e+05
9,cat__Store_3.0,-9.311898e+05,-9.311898e+05,-9.311898e+05


Sélection de variables : Le Lasso agit comme un sélecteur de variables. On observe que month et Store_19 sont ramenés à zéro, ce qui signifie qu’ils n’apportent aucune information utile pour prédire les ventes et sont donc éliminés.

Variables importantes : En revanche, certaines variables comme CPI, Unemployment ou encore les indicateurs liés aux Store conservent des coefficients importants (positifs ou négatifs), ainsi que month avec un facteur saisonnalité, indiquant qu’elles jouent un rôle majeur dans la prédiction des ventes.

Impact du paramètre alpha : On constate une faible fluctuation des coefficients entre alpha=1, 5 et 10. Cela suggère que la régularisation n’est pas encore assez forte pour forcer davantage de coefficients à zéro. Il serait pertinent de tester des valeurs plus élevées (50, 100, 500, ....) pour observer un effet plus marqué.



# Optimisation des Hyperparamètres
Comme nous avons pu le voir Ridge et Lasso agissent de façon différentes dans l'ajustement des features à l'aide de coefficient alpha. Afin de trouver le meilleur alpha pour chacun et d’obtenir des performances optimales, nous utiliserons une validation croisée via GridSearchCV. Cela permettra de tester plusieurs valeurs de alpha et de choisir celle qui minimise l’erreur sur les données de validation.

In [1837]:
# définitions des paramètres de Ridge
params = {'alpha': [0.01, 0.1, 1, 5, 10, 20, 50, 100 ]}

best_ridge = GridSearchCV(regressor_ridge, param_grid=params, cv=5)
best_ridge.fit(X_train,Y_train)
print("Best hyperparameters : ", best_ridge.best_params_)
print("Best R2 score : ", best_ridge.best_score_)


Best hyperparameters :  {'alpha': 0.1}
Best R2 score :  0.912124651300714


In [1838]:
# définitions des paramètres de Lasso
params = {'alpha': [1, 5, 10, 30, 50, 100, 200, 500, 1000, 2000]}

best_lasso = GridSearchCV(regressor_lasso, param_grid=params, cv=5)
best_lasso.fit(X_train,Y_train)
print("Best hyperparameters : ", best_lasso.best_params_)
print("Best R2 score : ", best_lasso.best_score_)

Best hyperparameters :  {'alpha': 500}
Best R2 score :  0.9131525868876288


#### Le meilleurs paramètre alpha pour 
- Ridge :  0.1
- Lasso : 500

## Évaluation et comparaison des modèles

- Prédire les ventes sur les jeux d’entraînement et de test.
- Calculer les performances avec le R² et la MAE pour évaluer précision et généralisation.
- Analyser les résidus en visualisant les erreurs (Résidus = Y_test - Y_test_pred) autour de zéro pour détecter biais ou anomalies.
- Comparer Ridge et Lasso afin d’identifier le modèle le plus fiable et le plus performant sur ce dataset.

In [1839]:
# Evaluons et comparaons nos modèles
def eval_model(model, X_train, X_test, Y_train, Y_test, model_name):
    # Prédictions
    Y_train_pred = model.predict(X_train)
    Y_test_pred = model.predict(X_test)
    
    # Calcul des métriques
    r2_train = r2_score(Y_train, Y_train_pred)
    r2_test = r2_score(Y_test, Y_test_pred)
    mae_train = mean_absolute_error(Y_train, Y_train_pred)
    mae_test = mean_absolute_error(Y_test, Y_test_pred)
    
    # Affichage des métriques
    print(f"\nRésultats pour {model_name}:")
    print(f"R² train: {r2_train:.3f}")
    print(f"R² test: {r2_test:.3f}")
    print(f"MAE train: {mae_train/1e6:.3f} millions $")
    print(f"MAE test: {mae_test/1e6:.3f} millions $")
    
   # Préparer les données pour le graphique en récupérant les resultats et en les mettant dans un dataframe
    df_residus = pd.DataFrame({
        "Prédictions": Y_test_pred,
        "Résidus": Y_test - Y_test_pred
    })
    print(df_residus.head(10))

    # Graphique 
    fig = px.scatter(
        df_residus,
        x="Prédictions",
        y="Résidus",
        title=f"Résidus vs Prédictions pour {model_name}",
        labels={"Prédictions":"Prédictions", "Résidus":"Résidus (Y_test - Y_test_pred)"},
        opacity=0.6
    )
    
    # Ligne horizontale à y=0
    fig.add_hline(y=0, line_dash="dash", line_color="red")
    
    fig.show()
   
# Évaluation
eval_model(best_ridge.best_estimator_, X_train, X_test, Y_train, Y_test, "RIDGE")
eval_model(best_lasso.best_estimator_, X_train, X_test, Y_train, Y_test, "LASSO")


Résultats pour RIDGE:
R² train: 0.972
R² test: 0.940
MAE train: 0.085 millions $
MAE test: 0.124 millions $
      Prédictions        Résidus
141  4.083720e+05 -106544.594258
13   1.428029e+06  249443.716064
29   1.599940e+06 -179534.205250
10   8.834041e+05   11662.422034
144  4.526627e+05  -28149.631938
70   1.091098e+06  497332.520200
79   2.047444e+06 -257756.712061
103  2.311436e+06   -8931.191425
33   2.076912e+06     344.016167
4    1.545879e+06   98592.007095



Résultats pour LASSO:
R² train: 0.972
R² test: 0.944
MAE train: 0.083 millions $
MAE test: 0.121 millions $
      Prédictions        Résidus
141  3.917663e+05  -89938.944138
13   1.426759e+06  250714.272488
29   1.600125e+06 -179719.598352
10   8.896571e+05    5409.431634
144  4.689148e+05  -44401.741750
70   1.122144e+06  466286.594320
79   2.036540e+06 -246852.615601
103  2.303200e+06    -694.838227
33   2.079907e+06   -2650.275307
4    1.547970e+06   96500.201076


## Analyse des coefficients des modèles

- Influence des variables sur les prédictions pour les trois modèles : régression linéaire simple, Ridge et Lasso.

- Récupération des coefficients après entraînement pour chaque modèle.

- Création d’un DataFrame regroupant les coefficients associés à chaque variable.

- Mise en forme pour visualisation : conversion en format “long” pour n'avoir que 3 colonnes Features, Modèle, Coef et tri des variables par importance absolue moyenne.


In [1840]:


# Récupération des coefficients
lin_coeffs = model.coef_
ridge_coeffs = best_ridge.best_estimator_.coef_
lasso_coeffs = best_lasso.best_estimator_.coef_

# Noms des features (après preprocessing)
feature_names = preprocessor.get_feature_names_out()

# Construction du DataFrame
df_coeffs = pd.DataFrame({
    "Feature": feature_names,
    "Linéaire": lin_coeffs,
    "Ridge": ridge_coeffs,
    "Lasso": lasso_coeffs
})

# Transformation en format "long" pour plotly express
df_long = df_coeffs.melt(id_vars="Feature", 
                         value_vars=["Linéaire","Ridge","Lasso"],
                         var_name="Modèle",
                         value_name="Coefficient")

# Tri par importance absolue moyenne
df_long["Importance"] = df_long["Coefficient"].abs()
order = df_long.groupby("Feature")["Importance"].mean().sort_values(ascending=False).index

# Graphique interactif
fig = px.bar(df_long, x="Coefficient", y="Feature", color="Modèle",
             title="Comparaison des coefficients (Linéaire, Ridge, Lasso)",
             barmode="group", category_orders={"Feature": order})

fig.update_layout(xaxis_title="Variables", yaxis_title="Valeur du coefficient", width=1200, height=900)
fig.show()


# Récapitulatif du performances:
| Modèle                     | R² train | R² test | MAE train  | MAE test  |
| -------------------------- | -------- | ------- | ---------- | --------- |
| Régression linéaire simple | 0.973    | 0.940   | 0.0836 M\$ | 0.125 M\$ |
| Ridge                      | 0.972    | 0.940   | 0.085 M\$  | 0.124 M\$ |
| Lasso                      | 0.972    | 0.944   | 0.083 M\$  | 0.121 M\$ |

#### > Observations

Tous les modèles expliquent plus de 94% de la variance sur le test → excellent ajustement global.

La MAE tourne autour de 0.12 M$, donc les prédictions restent très proches des ventes réelles.

Lasso obtient les meilleures performances globales (R² test = 0.944, MAE test = 0.121 M$), légèrement meilleur que Ridge et la régression linéaire simple.

### > Analyse des résidus

Les résidus sont globalement centrés autour de zéro → pas de biais systématique.

Quelques points extrêmes (> ±600k$) apparaissent, correspondant probablement à des semaines ou magasins atypiques (pics promotionnels, évènement local).

Comparaison Ridge vs Lasso :

Ridge et Régression linéaire simple donnent des résultats très similaires.

Lasso réduit légèrement certains écarts extrêmes, ce qui se reflète dans son meilleur score test.

### > Interprétation générale

Toutes les méthodes sont efficaces pour prédire Weekly_Sales.

Ridge → régularisation L2, réduit tous les coefficients uniformément → robustesse accrue.

Lasso → régularisation L1, permet de sélectionner certaines variables → utile si on veut simplifier le modèle.

Régression linéaire simple → performance très proche de Ridge et Lasso sur ce dataset, mais moins robuste si on ajoute du bruit ou des features corrélées.

### > Point d'amélioration

* Qualité et granularité des données

Les ventes sont très hétérogènes entre magasins.

Pour mieux prédire les pics et comprendre les variations, il serait utile de disposer de données plus régulières et détaillées par magasin, des informations hebdomadaires ou journalières supplémentaires, promotions, événements locaux,...

* Exploration de modèles non-linéaires

Les modèles linéaires (Ridge, Lasso, Régression linéaire simple) capturent bien la tendance générale, mais peuvent être limités face aux interactions complexes entre variables.

On pourrait tester des modèles non-linéaires comme :

Random Forest → capables de capturer des relations non-linéaires et interactions.

XGBoost → pour des performances optimisées sur des données tabulaires volumineuses.

* Feature engineering avancé

Créer des variables supplémentaires pour capturer la saisonnalité ( Nöel, Fêtes des mères...), les effets vacances, les promotions ( Soldes, BlackFriday, ..), ou les tendances spécifiques aux magasins pourrait améliorer les prédictions.

